## WIP 3D SIM reconstruction template

This is a work in progress template for running 3D SIM reconstruction based on the Janelia Python and c SIM code written by David Hoffman and Lin Shao

## 1.  Define code paths
Currently we hard code these and they need to be modified to run on different machines.  In the future we may move to a more intelligent approach like always having code exist beside the notebooks and using relative imports.  

In [ ]:
%pylab inline

import tifffile as tif
import os
import glob
%load_ext autoreload
%autoreload 
import shutil

# NOTE: BN the below code is from the legacy notebooks.  I am leaving it here for now, as it may be needed on some machines. 
if 'C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub' in sys.path:
    sys.path.remove('C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub')
else:
    pass

computer = 'bnort'

import sys

if computer == 'default':
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks')
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks\Scripts')
elif computer == 'bnort':
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts\Scripts')
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts')
else:
    pass

import dphutils 
from simrecon_utils import simrecon, split_process_recombine

# import dask
import dask
from dask.diagnostics import ProgressBar

# Setup home directory and OTF directory:

Right now we leave in paths for the current test machines but in the future may move to a more intelligent approach (for example user chooses paths with dialog box, paths stored in configuration)

Home should start from {Hesslab(\\prfs.hhmi.org)} e.g. home = r'Y:\Cryo_data2\ORCA_data\3D SIM Cells'

#### Additonal legacy notes (BN: these notes were in the original notebook I got) I'm leaving them in for now in case we need them for troubleshhoting

There was an old note "OTF folder should be placed inside Data processing notebooks"

(BN I don't think this has to be the case, because there are many notebook that define the full OTF path)

OTF Folder directory should be e.g.:   Y:\Cryo_data2\Data Processing Notebooks\SIM PSFs OTFs

Raw data from V-SIM data acquisition should be placed inside a dated folder here:   'Y:\Cryo_data2\ORCA_data\3D SIM Cells' 

with data file directory structure e.g.: 'Y:\Cryo_data2\ORCA_data\3D SIM Cells\20240322\488 nm 5 phases 0.81 NA Linear SIM_cam1_0.mrc'

In [ ]:
if computer == 'default': 
    home = r'Y:\Cryo_data2\488nm comparison TILED VS Non-Tiled at different base kwarg settings'
    otf_path = r'Y:\Seyforth\Data For Brian\Cryo-SIM Scope #2 Data (James System)\PSFs (best PSFs and examples of bad ones)\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'
elif computer == 'bnort':
    #home = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian'
    #home = r'D:\Janelia\Data 2024-10-02\560cm cell 4 _20240627_124604'
    home = r'D:\Janelia\Data 2024-10-10' 
    
    #otf_path = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\OTF\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'
    #otf_path = r'D:\Janelia\Data 2024-06-03\PSF-OTF used (Davids set of 4 wavelengths)\201909_19-20_best'
    #otf_path = r'C:\Users\bnort\work\Janelia\ims\OTF_folder'
OTFpath = os.path.join(otf_path,"*{}*.mrc")
OTFs = {wl : [path for path in glob.iglob(OTFpath.format(wl))] for wl in (560, 532, 488, 642)}
OTFs

## Set default params

Here we set the default params that will be used if none of them are overwritten.

Note that before calling the non-tiled and/or tiled processing code a small subset of parameters will be overwritten to values optimized for non-tiled/tiled processing

In [14]:
# David Solecki's suggestion: wiener=0.007 gammaApo=0.7 and suppressR=15
# original values by D.Hoffman: wiener=0.001 gammaApo=0.1 and suppressR=1.5

base_kwargs = dict(
                    nphases=5,
                    ndirs=3,
                    angle0= 1.36,
                    negDangle=True,              # James made False to try experiment
                    na= 0.85,
                    nimm= 1.0,
                    zoomfact= 2.0, 
                    background= 100.0,           # james experiment was 100.0
                    wiener= 0.007,
                    fastSIM=True,
                    otfRA= True,
                    dampenOrder0=True,
                    k0searchall=True,
                    equalizez=True,
                    preciseapo=True,
                    gammaApo=0.7,
                    suppressR=15.0
                )

def return_wl_otfs(path):
    if "488 Exc 532 Em" in path:
        wl = 532
    elif "488 Exc 642 Em" in path:
        wl = 642
    elif "532 Exc 561 Em" in path:
        wl = 561
    elif "560 nm" in path:
        wl = 560
    elif "488 nm" in path:
        wl = 488
    elif "532 nm" in path:
        wl = 532
    elif "642 nm" in path:
        wl = 642

    else:
        raise RuntimeError("no matching filename wavelength found, fix directory or filename or code")
    return wl, OTFs[wl]

## remove old processed data

This is commented out, but I assume we comment back in if we want to erase the previous run

In [ ]:
'''python
# clear processed
for path in glob.glob(home + "/*/*proc *.mrc"):
    os.remove(path)
for path in glob.glob(home + "/*/*proc *.txt"):
    os.remove(path)
'''

# Define process function defined to use DASK and simrecon package

This function is annotated as a dask ```delayed``` function.  Which means it will not be called right away but put in a queue to call using dask. 

In [16]:
@dask.delayed
def process(sim_kwargs, output_file_name):
    sim_output = simrecon(**sim_kwargs)
    with open(output_file_name.replace(".mrc", ".txt"), "w") as myfile:
        myfile.write(str(sim_kwargs))
        myfile.write("\n".join(sim_output))
    return output_file_name

# Cell below checks for raw data which has already been processed using full frame SIM reconstruction algorithm

In [ ]:
done_already = set(glob.iglob(home + "/**/*proc*.mrc", recursive = True))
done_already;
done_already = set()     # do this if want to re-process
print('Data processed already: \n')

for i in done_already:
    print(i + '\n')

## Set up parameters common between Non-tiled and tiled reconstruction

In [18]:


gammaApo = 0.3
suppressR = 1
wiener = 0.0001 

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener) 

nofilteroverlaps = False 
forcemodamp = False
forceotfamp = False

if nofilteroverlaps==True:
    user_text += '_nofilteroverlaps'

    if forcemodamp:
        o1 = 1.0
        o2 = 0.1
        user_text += '_forcemodamp_'+str(o1)+'_'+str(o2)
        forcemodamp = [o1, o2]

    if forceotfamp:
        o1 = 1
        o2 = 10
        user_text += '_forceotfamp_'+str(o1)+'_'+str(o2)
        forceotfamp = [o1, o2]
    
    base_kwargs.update(dict(nofilteroverlaps=nofilteroverlaps, gammaApo=gammaApo, suppressR=suppressR, wiener=wiener))   # default Full frame Recon. parameters    
    if forcemodamp:
        base_kwargs.update(dict(forcemodamp=forcemodamp))   # default Full frame Recon. parameters    
    elif forceotfamp:
        base_kwargs.update(dict(otfamp=forceotfamp))
                



# Full frame Simrecon SIM image reconstruction, recon parameters: 

Set up full frame processing.  Note that we override gammaApo, supressR and wiener parameters.  Also note the code will not be run right away (just put in a dask queue)

In [ ]:
to_process = []

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    wl, otfs = return_wl_otfs(raw)
    for otf in otfs:
        if "proc" not in raw:
            print('Data to process ' +str(raw))
            # Test with actually measured OTF
            sim_kwargs = dict(                                                                                                            
                input_file= raw,
                otf_file= otf,
                ls= (wl/1000)/2/0.81,)
            
            sim_kwargs.update(base_kwargs)
            OTF_filename = os.path.split(otf)[1].split('.')[0]
            print('\nOTF filename: ' +str(OTF_filename))
                
            #create processed file output name
            sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc", '_proc' + OTF_filename + '_' 
                                                                         + user_text + ".mrc")
                
            if sim_kwargs["output_file"] not in done_already:
                print("\n\nSIM recon Data to save:" + '\n' + str(sim_kwargs["output_file"]) + '\n')
                to_process.append(process(sim_kwargs, sim_kwargs["output_file"]))

# Running cell below shows all files being processed for full frame reconstruction and progress using DASK

This cell will start and run the processing.  We use dask to compute our processing queue that was set up in the cell above.

In [ ]:
with ProgressBar():
    print(to_process)
    out_names = done_already.update(*dask.compute(to_process))

# cell below checks for data already processed using the tiledreconstruction method, cell below that creates list of data to be set to done already, i.e. already processed


In [ ]:
for path in glob.glob(home + "/*/*_tile64_pad32*"):
    done_already.add(path.replace("_tile64_pad32", "") + "__split__")

[path for path in done_already if "_tile64_pad32" in path]

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    print(raw)

In [ ]:
print(home)
print(glob.glob(home + "/**/*SIM_*.mrc"))

# Cell below runs tiled reconstruction of SIM images

In [ ]:
#%%time

gammaApo = 0.2
suppressR =0.1
wiener = 0.001
done_already = set()
base_kwargs.update(dict(gammaApo=gammaApo, suppressR=suppressR, wiener=wiener)) # default tiling Recon. parameters

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener)

tile_size = 8
tile_overlap =4

filter_tiles = True
         
if filter_tiles:
    user_text=user_text+'_filter_tiles_set7'

    tile_limits = {}
   
    tile_limits['spacing_min'] = 0.31
    tile_limits['spacing_max'] = 0.33
    tile_limits['spacing_default']=0.315
    tile_limits['angle_min'] = 1.0
    tile_limits['angle_max'] = 2.0
    tile_limits['angle_default'] = 1.36
    tile_limits['amp1_min'] = 0.9
    tile_limits['amp1_max'] = 1.1
    tile_limits['amp1_default'] = 1.0
    tile_limits['amp2_min'] = 0.9
    tile_limits['amp2_max'] = 1.1
    tile_limits['amp2_default'] = 1.0
else:
    tile_limits = None



for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    wl, otfs = return_wl_otfs(raw)
    print(raw)
    
    for otf in otfs:
        if "proc" not in raw:                  #checks for if data was already processed
            if "_tile64_pad32" not in raw:       #checks for if data was already processed using tiling method
                print("\nFile to be processed: " + str(raw))
                # Test with actually measured OTF
                sim_kwargs = dict(
                    input_file= raw,
                    otf_file= otf,
                    #ls= (wl/1000)/2/0.81,
                    ls = .315
                )
                sim_kwargs.update(base_kwargs)
                
                OTF_filename = os.path.split(otf)[1].split('.')[0]
                
                #create processed file output name
                out_name = sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc",'_proc_' + OTF_filename + '_' +
                                                                          user_text + ".mrc")
                
                # perform reconstruction and output recon image file and text file
              
                out_name += "__split__"
                
                try:
                    if out_name not in done_already:
                        #print(sim_kwargs["input_file"])
                        sim_output = split_process_recombine(sim_kwargs["input_file"], tile_size, tile_overlap, sim_kwargs, tile_limits=tile_limits)
                        # HAD TO EDIT simrecon_utils.py lines 1488 and 1490 and comment out mrc.close() as it was failing.
                        with open(sim_output[0].replace(".mrc", ".txt"), "w") as myfile:
                            myfile.write(str(sim_kwargs))
                            myfile.write("\n" + "-" * 80 + "\n")
                            myfile.write("\n".join(sim_output[1]))

                    done_already.add(out_name)
                    for path in glob.glob(home + "/*/*_tile64_pad32*"):
                        done_already.add(path.replace("_tile64_pad32", "") + "__split__")
                except:
                    print('Error processing ' + str(raw))
                    continue

###### from simrecon_utils import process_txt_output, plot_params

In [ ]:
dst = "Tiled Parameters/"
try:
    os.mkdir(dst)
except FileExistsError:
    pass
for raw in glob.iglob(home + "/*/*SIM*tile*.txt"):
    with open(raw) as f:
        fig, axs = plot_params(process_txt_output("".join(f.readlines())))
        fig.savefig(dst + os.path.abspath(raw).split(os.path.sep)[-2] + ".png", dpi=300, bbox_inches="tight")

In [ ]:
print("done!")

# Renaming 3D SIM files for DropBox

In [ ]:
for wl in ("488 nm", "532 nm", "560 nm", "642 nm"):
    curdir = os.path.split(os.path.abspath(os.curdir))[1]
    glob_str = "F:/ORCA_Data/{}/3D SIM Cells/*{}*/*{}.mrc".format(curdir, "", "{}*SIM*proc*".format(wl))
    for path in sorted(glob.glob(glob_str)):
        print(path)
        path_head, path_tail = os.path.split(path)
        path_head_new, path_tail_new = os.path.split(path_head)
        file_root = path_tail_new.split("_")[0]+' '+path_tail_new.split("_")[1]

        OTF = path_tail.split('_OTF')[-1]
        proc = OTF.split('_proc')[-1]
        if proc == '.mrc':
            OTF = proc = OTF.split('_proc')[0] + '.mrc'
        new_path = os.path.join(path_head_new, file_root + ' wl'+wl.split(' ')[0]+' proc_with_OTF' + OTF)
        print(new_path)
        print('')
        
        shutil.copyfile(path,new_path)